In [124]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re

In [125]:
class FinancialStatements: 
    
    def getelementinlist(list,element):
        try:
            return list[element]
        except:
            return '-'
    
    def toInt(financialStatements):
        indexes = financialStatements.index
        columns = financialStatements.columns
        for column in columns:
            for index in indexes:
                aux = financialStatements[column][index]
                if re.search(r'-',aux):
                    aux1 = float(re.sub(r'-','0',aux))
                    financialStatements[column][index] = int(aux1)
                if re.search(r'B',aux):
                    if re.search(r'[()]',aux):
                        aux1 = float(re.sub(r'[B()]','',aux))*1000000000*-1
                        financialStatements[column][index] = int(aux1)
                    else:
                        aux1 = float(re.sub(r'[B]','',aux))*1000000000
                        financialStatements[column][index] = int(aux1)
                if re.search(r'M',aux):
                    if re.search(r'[()]',aux):
                        aux1 = float(re.sub(r'[M()]','',aux))*1000000*-1
                        financialStatements[column][index] = int(aux1)
                    else:
                        aux1 = float(re.sub(r'[M]','',aux))*1000000
                        financialStatements[column][index] = int(aux1)
        return financialStatements
    
    def getFinancialStatements(self,Stocks, dates, root, folder, form):
        i = 0
        no_of_stocks = len(Stocks.index)
        for i in range(5):
            try:
                urlfinancials = 'https://www.marketwatch.com/investing/stock/'+ Stocks.iloc[i]['Symbols'] +'/financials'
                urlbalancesheet = 'https://www.marketwatch.com/investing/stock/'+ Stocks.iloc[i]['Symbols'] +'/financials/balance-sheet'
                urlcashflow = 'https://www.marketwatch.com/investing/stock/'+ Stocks.iloc[i]['Symbols'] +'/financials/cash-flow'
                text_soup_financials = BeautifulSoup(requests.get(urlfinancials).text,"lxml")
                text_soup_balancesheet = BeautifulSoup(requests.get(urlbalancesheet).text,"lxml")
                text_soup_cashflow = BeautifulSoup(requests.get(urlcashflow).text,"lxml")

                # Income Statement
                titlesfinancials = text_soup_financials.findAll('td', {'class': 'rowTitle'})
                saleslist = []
                grossincomelist = []
                pretaxincomelist = []
                netincomelist = []
                ebitdalist= []
                for title in titlesfinancials:
                    if 'Sales/Revenue' in title.text:
                        saleslist.append ([td.text for td in title.findNextSiblings(attrs={'class': 'valueCell'}) if td.text])
                    if 'Gross Income' in title.text:
                        grossincomelist.append ([td.text for td in title.findNextSiblings(attrs={'class': 'valueCell'}) if td.text])
                    if 'Pretax Income' in title.text:
                        pretaxincomelist.append ([td.text for td in title.findNextSiblings(attrs={'class': 'valueCell'}) if td.text])
                    if 'Net Income' in title.text:
                        netincomelist.append ([td.text for td in title.findNextSiblings(attrs={'class': 'valueCell'}) if td.text])
                    if 'EBITDA' in title.text:
                        ebitdalist.append ([td.text for td in title.findNextSiblings(attrs={'class': 'valueCell'}) if td.text])

                # Balance sheet
                titlesbalancesheet = text_soup_balancesheet.findAll('td', {'class': 'rowTitle'})
                cashshortlist = []
                totalassetslist = []
                totalcurrentliabilitieslist = []
                totalliabilitieslist = [] 
                totalequitylist = []
                for title in titlesbalancesheet:
                    if 'Cash & Short Term Investments' in title.text:
                        cashshortlist.append ([td.text for td in title.findNextSiblings(attrs={'class': 'valueCell'}) if td.text])
                    if 'Total Assets' in title.text:
                        totalassetslist.append ([td.text for td in title.findNextSiblings(attrs={'class': 'valueCell'}) if td.text])
                    if 'Total Current Liabilities' in title.text:
                        totalcurrentliabilitieslist.append ([td.text for td in title.findNextSiblings(attrs={'class': 'valueCell'}) if td.text])
                    if 'Total Liabilities' in title.text:
                        totalliabilitieslist.append ([td.text for td in title.findNextSiblings(attrs={'class': 'valueCell'}) if td.text])
                    if 'Total Equity' in title.text:
                        totalequitylist.append ([td.text for td in title.findNextSiblings(attrs={'class': 'valueCell'}) if td.text])

                # Cash Flow
                titlescashflow = text_soup_cashflow.findAll('td', {'class': 'rowTitle'})
                netoperatingcashflowlist = []
                netinvestingcashflowlist = []
                netfinincingcashflowlist = []
                freecashflowlist = []
                for title in titlescashflow:
                    if 'Net Operating Cash Flow' in title.text:
                        netoperatingcashflowlist.append ([td.text for td in title.findNextSiblings(attrs={'class': 'valueCell'}) if td.text])
                    if 'Net Investing Cash Flow' in title.text:
                        netinvestingcashflowlist.append ([td.text for td in title.findNextSiblings(attrs={'class': 'valueCell'}) if td.text])
                    if 'Net Financing Cash Flow' in title.text:
                        netfinincingcashflowlist.append ([td.text for td in title.findNextSiblings(attrs={'class': 'valueCell'}) if td.text])
                    if 'Free Cash Flow' in title.text:
                        freecashflowlist.append ([td.text for td in title.findNextSiblings(attrs={'class': 'valueCell'}) if td.text])

                # Income Statement
                sales = FinancialStatements.getelementinlist(saleslist,0)
                grossincome = FinancialStatements.getelementinlist(grossincomelist,0)
                pretaxincome = FinancialStatements.getelementinlist(pretaxincomelist,0)
                netincome = FinancialStatements.getelementinlist(netincomelist,0)
                ebitda = FinancialStatements.getelementinlist(ebitdalist,0)

                #Balance Sheet
                cashshort = FinancialStatements.getelementinlist(cashshortlist,0)
                totalassets = FinancialStatements.getelementinlist(totalassetslist,1)
                totalcurrentliabilities = FinancialStatements.getelementinlist(totalcurrentliabilitieslist,0)
                totalliabilities = FinancialStatements.getelementinlist(totalliabilitieslist,0)
                totalequity = FinancialStatements.getelementinlist(totalequitylist,0)

                #Cash Flow
                netoperatingcashflow = FinancialStatements.getelementinlist(netoperatingcashflowlist,0)
                netinvestingcashflow = FinancialStatements.getelementinlist(netinvestingcashflowlist,0)
                netfinancingcashflow = FinancialStatements.getelementinlist(netfinincingcashflowlist,0)
                freecashflow = FinancialStatements.getelementinlist(freecashflowlist,0)

                financialStatements= pd.DataFrame({'sales': sales,'grossincome': grossincome, 'ebitda': ebitda, 
                                                     'pretaxincome': pretaxincome,'netincome': netincome, 'cashshort': cashshort, 
                                                     'totalassets': totalassets,'totalcurrentliabilities': totalcurrentliabilities, 'totalliabilities': totalliabilities,
                                                     'totalequity': totalequity, 'netoperatingcashflow': netoperatingcashflow, 
                                                     'netinvestingcashflow': netinvestingcashflow, 'netfinancingcashflow': netfinancingcashflow,  
                                                     'freecashflow': freecashflow}, index = dates)
                financial_statements = FinancialStatements.toInt(financialStatements)
                financial_statements.to_csv(root + folder + Stocks.iloc[i]['Symbols']+ 'FinancialStatements' + form, index = True, header=True)
            except:
                print('An error occurred on ' + Stocks.iloc[i]['Symbols'])

In [126]:
Stocks = pd.read_csv('./Stocks.csv')
dates = [2015,2016,2017,2018,2019]
root = './Stocks/'
folder = ''
form = '.csv'

In [127]:
financialStatements = FinancialStatements()
financialStatements.getFinancialStatements(Stocks, dates, root, folder, form)